In [ ]:
from nutils import mesh, function, solver, export, cli, testing
import numpy, treelog
from matplotlib import pyplot as plt

timescale=0.5
newtontol=1e-5
endtime=0.5
nelems=10

domain, geom = mesh.rectilinear([numpy.linspace(0,1,nelems+1)])

ns = function.Namespace()
ns.p = geom
ns.fbasis = domain.basis('std', degree=1)
ns.f = 'fbasis_n ?flhs_n'
ns.Dbasis = domain.basis('discont', degree=0)
ns.D = 'Dbasis_n ?Dlhs_n'

#set D as a time independent coefficient D_0
Dsqr = domain.integral('( D - ((p_0 - 0.5)^2 + 1))^2 J(p)' @ ns, degree=5)
Dlhs0 = solver.optimize('Dlhs', Dsqr)

res=domain.integral('-D d(f, p_i) d(fbasis_n, p_i) J(p)' @ ns, degree=5)
inertia = domain.integral('fbasis_n f J(p)' @ ns, degree=5)

#set initial value of f
fsqr=domain.integral('( f - (-(p_0 - 0.5)^2 + 1))^2 J(p)' @ ns, degree=5)
flhs0=solver.optimize('flhs',fsqr)

timestep = timescale/nelems
with treelog.iter.plain('timestep', solver.impliciteuler('flhs', res, inertia, timestep=timestep, arguments=dict(flhs=flhs0, Dlhs=Dlhs0), newtontol=newtontol)) as steps:
    for itime, flhs in enumerate(steps):
      if itime * timestep >= endtime:
        break

    
bezier = domain.sample('bezier', 32)
nanjoin = lambda array, tri: numpy.insert(array.take(tri.flat, 0).astype(float), slice(tri.shape[1], tri.size, tri.shape[1]), numpy.nan, axis=0)
sampled_x = nanjoin(bezier.eval(ns.p[0]), bezier.tri)
def plot_line(func, **arguments):
    plt.plot(sampled_x, nanjoin(bezier.eval(func, **arguments), bezier.tri))
    plt.xlabel('p_0')
    plt.xticks(numpy.linspace(0, 1, 5))

plot_line(ns.D, Dlhs=Dlhs0)